In [1]:
import urllib
import requests
import pandas as pd
import json
import base64

In [2]:
text = 'david :%&= silva'
urllib.parse.quote_plus(text)

'david+%3A%25%26%3D+silva'

In [3]:
query = 'artist:silva genre:pop tag:new'
url_query = urllib.parse.quote_plus(query)

In [4]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                   encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url,
                            data = data,
                            headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [5]:
keys = pd.read_csv(r"../..\Dropbox\Kent\EMAT-32210_Data-in-EMAT\Spotify-Keys_9-12-23.txt")

In [6]:
# keys

In [7]:
access_token = get_session_token(keys['Client_ID'].iloc[0], keys['Client_Secret'].iloc[0])

200


In [8]:
access_token

'BQAx_bQa_GK5NvxhLuz3gaiPpUbpDQPGaPAXM2JDGsbiEFDEcKG31Lu_vMNM-gNXZogwN_iolagqL6xFnwO8lGHvd4spw2La_DetPkMm67Uv7O22kso'

In [9]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
tracks_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

In [10]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [11]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [12]:
sc_album_id = '3iPSVi54hsacKKl1xIR2eH'
cr_album_id = '0EiI8ylL0FmWWpgHVTsZjZ'

In [13]:
ab_tracks_ep.format(sc_album_id)

'https://api.spotify.com/v1/albums/3iPSVi54hsacKKl1xIR2eH/tracks'

In [14]:
sc_album_response = api_call(ab_tracks_ep.format(sc_album_id), session_header)
cr_album_response = api_call(ab_tracks_ep.format(cr_album_id), session_header)

200
200


In [15]:
sc_album_df = pd.DataFrame(sc_album_response['items'])
cr_album_df = pd.DataFrame(cr_album_response['items'])
# sc_album_df.head()

In [16]:
sc_track_ids = ','.join(sc_album_df['id'].to_list())
cr_track_ids = ','.join(cr_album_df['id'].to_list())

In [17]:
t_features_ep + '?ids={}'.format(sc_track_ids)

'https://api.spotify.com/v1/audio-features?ids=5G2f63n7IPVPPjfNIGih7Q,2tHwzyyOLoWSFqYNjeVMzj,102YUQbYmwdBXS7jwamI90,3iPIDAFybaoyqX7hvAfWkl,5oIVNm56t6OIf9ZjdEG3ud,1UHS8Rf6h5Ar3CDWRd3wjF,2HRqTpkrJO5ggZyyK6NPWz,5VhvD0AKRprEaFtPNKGBQR,0XkZmBCCcdMY0EPY8ij6Gb,21B4gaTWnTkuSh77iWEXdS,5il0jwWUlvgtIzWvzJi12z,09LrGvT9KsACH66RHYMDyR'

In [18]:
sc_track_features = api_call(t_features_ep + '?ids={}'.format(sc_track_ids),
                             session_header)
cr_track_features = api_call(t_features_ep + '?ids={}'.format(cr_track_ids),
                             session_header)

200
200


In [19]:
sc_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(sc_track_ids),
                             session_header)
cr_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(cr_track_ids),
                             session_header)

200
200


In [20]:
sc_features_df = pd.DataFrame(sc_track_features['audio_features'])
cr_features_df = pd.DataFrame(cr_track_features['audio_features'])
# sc_features_df.head()

In [21]:
sc_tracks_df = pd.DataFrame(sc_track_info['tracks'])
cr_tracks_df = pd.DataFrame(cr_track_info['tracks'])
sc_tracks_df.head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,157279,False,{'isrc': 'USUM72404100'},{'spotify': 'https://open.spotify.com/track/5G...,https://api.spotify.com/v1/tracks/5G2f63n7IPVP...,5G2f63n7IPVPPjfNIGih7Q,False,True,Taste,96,1,track,spotify:track:5G2f63n7IPVPPjfNIGih7Q
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,186365,True,{'isrc': 'USUM72404101'},{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tHwzyyOLoWS...,2tHwzyyOLoWSFqYNjeVMzj,False,True,Please Please Please,84,2,track,spotify:track:2tHwzyyOLoWSFqYNjeVMzj
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,185265,True,{'isrc': 'USUM72404102'},{'spotify': 'https://open.spotify.com/track/10...,https://api.spotify.com/v1/tracks/102YUQbYmwdB...,102YUQbYmwdBXS7jwamI90,False,True,Good Graces,89,3,track,spotify:track:102YUQbYmwdBXS7jwamI90
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,218284,True,{'isrc': 'USUM72404104'},{'spotify': 'https://open.spotify.com/track/3i...,https://api.spotify.com/v1/tracks/3iPIDAFybaoy...,3iPIDAFybaoyqX7hvAfWkl,False,True,Sharpest Tool,86,4,track,spotify:track:3iPIDAFybaoyqX7hvAfWkl
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,164202,False,{'isrc': 'USUM72405667'},{'spotify': 'https://open.spotify.com/track/5o...,https://api.spotify.com/v1/tracks/5oIVNm56t6OI...,5oIVNm56t6OIf9ZjdEG3ud,False,True,Coincidence,86,5,track,spotify:track:5oIVNm56t6OIf9ZjdEG3ud


In [23]:
sc_merged = pd.merge(sc_features_df, sc_tracks_df,
                     how = 'inner', on = 'id')
sc_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_ids,external_urls,href,is_local,is_playable,name,popularity,track_number,type_y,uri_y
0,0.674,0.907,3,-4.086,1,0.0640,0.101,0.000000,0.2970,0.721,...,{'isrc': 'USUM72404100'},{'spotify': 'https://open.spotify.com/track/5G...,https://api.spotify.com/v1/tracks/5G2f63n7IPVP...,False,True,Taste,96,1,track,spotify:track:5G2f63n7IPVPPjfNIGih7Q
1,0.675,0.586,9,-6.086,1,0.0531,0.257,0.000000,0.1040,0.621,...,{'isrc': 'USUM72404101'},{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tHwzyyOLoWS...,False,True,Please Please Please,84,2,track,spotify:track:2tHwzyyOLoWSFqYNjeVMzj
2,0.758,0.759,5,-5.315,1,0.0479,0.329,0.000004,0.1440,0.732,...,{'isrc': 'USUM72404102'},{'spotify': 'https://open.spotify.com/track/10...,https://api.spotify.com/v1/tracks/102YUQbYmwdB...,False,True,Good Graces,89,3,track,spotify:track:102YUQbYmwdBXS7jwamI90
3,0.716,0.630,4,-7.509,1,0.1080,0.475,0.000000,0.0943,0.588,...,{'isrc': 'USUM72404104'},{'spotify': 'https://open.spotify.com/track/3i...,https://api.spotify.com/v1/tracks/3iPIDAFybaoy...,False,True,Sharpest Tool,86,4,track,spotify:track:3iPIDAFybaoyqX7hvAfWkl
4,0.418,0.815,9,-6.216,1,0.1250,0.362,0.000000,0.1250,0.758,...,{'isrc': 'USUM72405667'},{'spotify': 'https://open.spotify.com/track/5o...,https://api.spotify.com/v1/tracks/5oIVNm56t6OI...,False,True,Coincidence,86,5,track,spotify:track:5oIVNm56t6OIf9ZjdEG3ud


In [24]:
sc_merged.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'name', 'popularity', 'track_number', 'type_y', 'uri_y'],
      dtype='object')

In [26]:
cr_merged = pd.merge(cr_features_df, cr_tracks_df,
                     how = 'inner', on = 'id')
sc_merged['artist'] = "Sabrina Carpenter"
cr_merged['artist'] = "Chappell Roan"
sc_merged.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'name', 'popularity', 'track_number', 'type_y', 'uri_y',
       'artist'],
      dtype='object')

In [30]:
artist_compare = pd.concat([sc_merged, cr_merged], axis = 0)
artist_compare

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,external_urls,href,is_local,is_playable,name,popularity,track_number,type_y,uri_y,artist
0,0.674,0.907,3,-4.086,1,0.0640,0.10100,0.000000,0.2970,0.721,...,{'spotify': 'https://open.spotify.com/track/5G...,https://api.spotify.com/v1/tracks/5G2f63n7IPVP...,False,True,Taste,96,1,track,spotify:track:5G2f63n7IPVPPjfNIGih7Q,Sabrina Carpenter
1,0.675,0.586,9,-6.086,1,0.0531,0.25700,0.000000,0.1040,0.621,...,{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tHwzyyOLoWS...,False,True,Please Please Please,84,2,track,spotify:track:2tHwzyyOLoWSFqYNjeVMzj,Sabrina Carpenter
2,0.758,0.759,5,-5.315,1,0.0479,0.32900,0.000004,0.1440,0.732,...,{'spotify': 'https://open.spotify.com/track/10...,https://api.spotify.com/v1/tracks/102YUQbYmwdB...,False,True,Good Graces,89,3,track,spotify:track:102YUQbYmwdBXS7jwamI90,Sabrina Carpenter
3,0.716,0.630,4,-7.509,1,0.1080,0.47500,0.000000,0.0943,0.588,...,{'spotify': 'https://open.spotify.com/track/3i...,https://api.spotify.com/v1/tracks/3iPIDAFybaoy...,False,True,Sharpest Tool,86,4,track,spotify:track:3iPIDAFybaoyqX7hvAfWkl,Sabrina Carpenter
4,0.418,0.815,9,-6.216,1,0.1250,0.36200,0.000000,0.1250,0.758,...,{'spotify': 'https://open.spotify.com/track/5o...,https://api.spotify.com/v1/tracks/5oIVNm56t6OI...,False,True,Coincidence,86,5,track,spotify:track:5oIVNm56t6OIf9ZjdEG3ud,Sabrina Carpenter
5,0.739,0.727,11,-5.968,0,0.0426,0.06780,0.000000,0.1040,0.676,...,{'spotify': 'https://open.spotify.com/track/1U...,https://api.spotify.com/v1/tracks/1UHS8Rf6h5Ar...,False,True,Bed Chem,92,6,track,spotify:track:1UHS8Rf6h5Ar3CDWRd3wjF,Sabrina Carpenter
6,0.699,0.776,0,-5.282,1,0.0293,0.13100,0.000005,0.2050,0.708,...,{'spotify': 'https://open.spotify.com/track/2H...,https://api.spotify.com/v1/tracks/2HRqTpkrJO5g...,False,True,Espresso,83,7,track,spotify:track:2HRqTpkrJO5ggZyyK6NPWz,Sabrina Carpenter
7,0.471,0.389,7,-5.421,1,0.0264,0.77200,0.000000,0.2470,0.238,...,{'spotify': 'https://open.spotify.com/track/5V...,https://api.spotify.com/v1/tracks/5VhvD0AKRprE...,False,True,Dumb & Poetic,83,8,track,spotify:track:5VhvD0AKRprEaFtPNKGBQR,Sabrina Carpenter
8,0.619,0.542,2,-8.681,1,0.0529,0.28800,0.000000,0.1510,0.308,...,{'spotify': 'https://open.spotify.com/track/0X...,https://api.spotify.com/v1/tracks/0XkZmBCCcdMY...,False,True,Slim Pickins,84,9,track,spotify:track:0XkZmBCCcdMY0EPY8ij6Gb,Sabrina Carpenter
9,0.633,0.725,5,-4.257,1,0.0427,0.04610,0.000000,0.2050,0.640,...,{'spotify': 'https://open.spotify.com/track/21...,https://api.spotify.com/v1/tracks/21B4gaTWnTku...,False,True,Juno,88,10,track,spotify:track:21B4gaTWnTkuSh77iWEXdS,Sabrina Carpenter
